# Implement The Linear Regression ML Models




In [ ]:
# libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# import
df = pd.read_csv(r"Cleaned_data.csv")
df.head()

In [ ]:
## Model Building
# 1. select the numerical columns
# 2. seprate the x and y
# 2.1 split the data into training testing
# 3. load the algorithms and create the object of algoritms
# 4. train the model --> find the pattern value from data
# 5. test then model/predict
# 6. Evaluate/accuracy the model

# 7. fine tuning the model(for improve accuracy)

### 1. with only numerical columns

In [ ]:
# 1. select the numerical columns
df = df.drop(columns=["area_type","location"], axis=1)
df.head()

In [ ]:
# 2. seprate the x and y
X = df.drop(columns=["price"])
y = df["price"]

In [ ]:
y

In [ ]:
# 2. split the data into training testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state =0)

print(X_train.shape)
print(X_test.shape)

In [ ]:
# 3. load the algorithms and create the object of algoritms
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [ ]:
# 4. train the model --> find the pattern value from data
lr.fit(X_train,y_train)

In [ ]:
# coffecient, intercept
print(lr.coef_)
print(lr.intercept_)

In [ ]:
# test
y_pred = lr.predict(X_test)
y_pred

In [ ]:
# R2score for check variance
r2_score(y_test,y_pred)

### 2. with categorical and numerical columns with pipeline Concept

In [ ]:
# import
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotStandardScalerEncoder

In [ ]:
# train test split
from sklearn.model_selection import train_test_split

X = df.drop(columns=["price"])
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# task 1 transform applying
# columns transform
columns_trans = ColumnTransformer(
    [('onehot_location', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ['location']),
     ('onehot_area_type', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ["area_type"]),
     ('scaler', StandardScaler(), ["total_sqft", "bath"]),

     ],
    remainder='passthrough')
# task 2 model a
# model
lr = LinearRegression()

#pipeline
pipe = make_pipeline(columns_trans,lr)

print(pipe)

In [ ]:
# Train
pipe.fit(X_train,y_train)

# Predictions
y_pred = pipe.predict(X_test)
y_pred

array([155.50296562,  92.4472685 , 102.94028748, ..., 169.02451684,
       106.82742062, 128.69041058])

# Performance Matrix
Measuring Performance metrics-Lost and Cost Function (MAE,MSE,RMSE,R2 Score)

In [ ]:
# cost functions --> calculate erros
from sklearn.metrics import mean_absolute_error, mean_squared_error,root_mean_squared_error

print("MAE:",mean_absolute_error(y_test,y_pred))
print("MSE:",mean_squared_error(y_test,y_pred))
print("RMSE:",root_mean_squared_error(y_test,y_pred))

In [ ]:
# Regression Performance check using r2_score ,and Adjusted r2 score
from sklearn.metrics import r2_score
r_squared = r2_score(y_test,y_pred)
r_squared

# https://benjaminobi.medium.com/what-really-is-r2-score-in-linear-regression-20cafdf5b87c

In [ ]:
# adjusted_r2
n_samples = df.shape[0]
n_features = df.shape[1]

In [ ]:
adjusted_r2 = 1 - (1 - r_squared) * (n_samples - 1) / (n_samples - n_features - 1)
adjusted_r2
# https://www.shiksha.com/online-courses/articles/adjusted-r-squared/
# https://benjaminobi.medium.com/what-really-is-r2-score-in-linear-regression-20cafdf5b87c

In [ ]:
# save the movel for future use
import pickle
pickle.dump(pipe,open("model.pkl","wb"))

In [ ]:
locations = df["location"].unique()
# save the data for future use
pickle.dump(locations, open("locations.pkl","wb"))

# carpet area
carpet_area = df["area_type"].unique()
# save the data for future use
pickle.dump(carpet_area, open("area_type.pkl","wb"))

In [ ]:
# load the model
pipe = pickle.load(open("model.pkl","rb"))

In [ ]:
# # load the model
# pipe = pickle.load(open("model.pkl","rb"))

# # take input from user
# location = input("Enter location: ")
# area_type = input("Enter area type: ")
# total_sqft = float(input("Enter total square feet: "))
# bath = float(input("Enter number of bathrooms: "))
# bhk = int(input("Enter number of bedrooms (BHK): "))
# balcony = int(input("Enter number of balcony: "))

# # Create a DataFrame from user input
# user_input = pd.DataFrame([[location, area_type, total_sqft, bath, bhk,balcony]],
#                           columns=['location', 'area_type', 'total_sqft', 'bath', 'bhk',"balcony"])

# # Predict the price
# predicted_price = pipe.predict(user_input)

# print(f"The predicted price is: {predicted_price[0]:.2f} Lakhs")
# # Electronic City Phase II

### Save the Streamlit app code to app.py

In [ ]:
# with Dropdown meanu use
import streamlit as st
import pandas as pd
import pickle

# ----------------------
# Load Model & Dropdown Data
# ----------------------
pipe = pickle.load(open("model.pkl", "rb"))
locations = pickle.load(open("locations.pkl", "rb"))
area_types = pickle.load(open("area_type.pkl", "rb"))

# ----------------------
# Streamlit App
# ----------------------
st.set_page_config(page_title="House Price Prediction", layout="centered")
st.title("🏠 House Price Prediction App")
st.write("Enter property details below to get an estimated price (in Lakhs).")

# User inputs
location = st.selectbox("Select Location", locations)
area_type = st.selectbox("Select Area Type", area_types)
total_sqft = st.number_input("Enter Total Square Feet", min_value=100.0, step=10.0)
bath = st.number_input("Enter Number of Bathrooms", min_value=1.0, step=1.0)
bhk = st.number_input("Enter Number of Bedrooms (BHK)", min_value=1, step=1)
balcony = st.number_input("Enter number of Balcony: ", min_value=0)


# Prediction button
if st.button("Predict Price"):
    # Create DataFrame from inputs
    user_input = pd.DataFrame([[location, area_type, total_sqft, bath, bhk,balcony]],
                          columns=['location', 'area_type', 'total_sqft', 'bath', 'bhk','balcony'])

    # Make prediction
    predicted_price = pipe.predict(user_input)[0]

    st.success(f"💰 Estimated Price: **{predicted_price:.2f} Lakhs**")


In [ ]:
# steps
# 1. take all pkl files in one folder after training
# 2. create aap.py --> insert all code
# 3. create requirements.txt --> write all library which is needed to run project



In [ ]:
# For model improvement check relationship 
# and change models to improvement

In [ ]:
# Regression models --> liner---> LinearRegression, ridge lasso, SGDRegressor ,
#       non-linear -->(Decision tree) DecisionTreeRegressor, RandomForestRegressor,
#       ensembles models --> GradientBoostingRegressor , support_vector_regression

# classifications ---> Logistic regression(linear)
# (non-linear) SVM(SVC), KNN,naive bayes, Decision tree(classifier),RandomForestClassifer,
#       ensembles models --> all



# Deeplearning --> non-linear -Neural Network (only change in architecture for diffrent work)

### Topics
- Assumptions of Machine Learning(all ready done)
- Simple linear regression
- multiple linear regression
- variant of LR for RIDGE, LASSO ,ELASTICNET
- Measuring Performance metrics-Lost and Cost Function Model Evaluation Metrics ( MSE, MAE, RMSE)(R²-score, Adjusted R²-score)

variant of Linear models
- Polynomial Regression
- Linear Regression with OLS
- Linear Regression with SGD

- Under fitting and Overfitting
- Regularization ,Lasso & Ridge


# 1. assumption In Linear Regression

- Homoscedasticity

The variance of residuals should be approximately equal for all predicted values of the dependent variable.

- Normality - Errors are normally distributed

The residuals should be normally distributed.Linear relationship

There is a linear relationship between the dependent and independent variables

The error terms are normally distributed and the data do not contain outliers

- Independence of errors

The residual errors are independent of each other.

- No or low multicollinearity

There is no or little collinearity among input variables

#
https://www.geeksforgeeks.org/machine-learning/assumptions-of-linear-regression/
https://www.kdnuggets.com/2021/02/machine-learning-assumptions.html

![](https://media.geeksforgeeks.org/wp-content/uploads/20241025105428904464/Assumptions-of-Linear-Regression.webp)


<!-- #### MultiColiniarity
(optional)

- What it is

Multicollinearity occurs when independent variables are correlated, which can make it difficult to estimate each variable's relationship with the outcome variable.

- Why it's a problem

Multicollinearity can negatively impact model predictions on unseen data, and can make it difficult to explain the model's behavior.

- How to detect it

A correlation matrix can help identify multicollinearity, but a heatmap of correlations can be more intuitive. A variance inflation factor (VIF) can also be used to measure the amount of multicollinearity.

- How to address it

If multicollinearity is detected, it can be addressed by using a VIF to identify the correlated variables, and then removing or transforming those variables


from sklearn.datasets import fetch_california_housing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = fetch_california_housing()
df = pd.DataFrame(data.data, columns = data.feature_names)
df['Price'] = data.target


#multicollinearity>> when one feature is explained by all other features
#when two features>> correlation
#more than two features>> vif, clustermap

sns.heatmap(df.corr(), annot = True, vmin = -1, vmax = 1)

#clustermap table shows group(cluster ) relationship
plt.figure(figsize = (5, 5))
sns.clustermap(df.corr(), vmin = -1, vmax =1, annot = True)

### What is the function of variance_inflation_factor?
Variance Inflation Factor (VIF)

A variance inflation factor (VIF) is a measure of the amount of multicollinearity in regression analysis. Multicollinearity exists when there is a correlation between multiple independent variables in a multiple regression model. This can adversely affect the regression results.


from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif['Feature'] = df.columns
vif

vif["VIF"] = [variance_inflation_factor(df.values, i) for i in range(len(df.columns))]

df1 = df.copy()

df1.drop("Longitude", axis=1, inplace=True)
df1
vif = pd.DataFrame()
vif['Feature'] = df1.columns
vif["VIF"] = [variance_inflation_factor(df1.values, i) for i in range(len(df1.columns))]

vif

df1.drop("AveRooms", axis=1, inplace=True)
vif = pd.DataFrame()
vif['Feature'] = df1.columns
vif["VIF"] = [variance_inflation_factor(df1.values, i) for i in range(len(df1.columns))]

df1.drop("Latitude", axis=1, inplace=True)
vif = pd.DataFrame()
vif['Feature'] = df1.columns
vif["VIF"] = [variance_inflation_factor(df1.values, i) for i in range(len(df1.columns))]

Recursive Feature Elimination (RFE) is a feature selection method that iteratively removes features and trains a model on the remaining ones, evaluating its performance at each step. By repeatedly eliminating the least important features, RFE identifies the features that contribute most to the model's accuracy. It's a valuable tool for simplifying models and improving their efficiency.

#RFE >> recruseive feature elimination
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

from sklearn.feature_selection import RFE

rfe = RFE(estimator = LinearRegression(), n_features_to_select=6)

rfe.fit(X, y)

rfe.predict(X)

rfe.support_

X.columns

rfe.ranking_

 -->
rivision codes inside this markdown

## 2. Simple Linear Regression

- y=mx +c
single feature use

## 3. Multiple Linear Regression

- multiple  feature/X use
- y =M1x1 +M2x2 + M3x3 + MnXn +c

## 4. Ridge Regression

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
# Ridge Regression
from sklearn.linear_model import Ridge
# pipe_lr = make_pipeline(columns_trans, lr)
pipe_ridge = make_pipeline(columns_trans, Ridge(alpha=1.0))

pipe_ridge.fit(X_train, y_train)
y_pred_ridge = pipe_ridge.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)


print(f"Ridge Regression MSE: {mse_ridge:.4f}")

Ridge Regression MSE: 11184.5695


In [13]:
# Lasso Regression
from sklearn.linear_model import Lasso
pipe_lasso = make_pipeline(columns_trans, Lasso(alpha=0.1))


pipe_lasso.fit(X_train, y_train)
y_pred_lasso = pipe_lasso.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)

print(f"Lasso Regression MSE: {mse_lasso:.4f}")

Lasso Regression MSE: 11665.5722


In [14]:
# Elastic Net Regression
from sklearn.linear_model import ElasticNet
pipe_elastic_net = make_pipeline(columns_trans, ElasticNet(alpha=0.1, l1_ratio=0.5))


pipe_elastic_net.fit(X_train, y_train)
y_pred_elastic_net = pipe_elastic_net.predict(X_test)
mse_elastic_net = mean_squared_error(y_test, y_pred_elastic_net)

print(f"Elastic Net Regression MSE: {mse_elastic_net:.4f}")

Elastic Net Regression MSE: 12471.4969
